In [2]:
pip install kafka-python

Note: you may need to restart the kernel to use updated packages.


In [21]:
!pip install alpha-vantage

In [23]:
from alpha_vantage.timeseries import TimeSeries

In [25]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json
import requests
import time

In [27]:
producer = KafkaProducer(bootstrap_servers=['18.119.101.110:9092'], 
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [29]:
producer.send('demo_test', value={'surnasdasdame':'abhiramk'})

In [31]:
API_KEY = 'U1CJJD8MTIGOXDHF'  # Replace with your Alpha Vantage API key
SYMBOL = 'IBM'           # Stock symbol to track
INTERVAL = '5min'        # Time interval (1min, 5min, 15min, 30min, 60min)
OUTPUT_SIZE = 'compact'  # 'compact' (last 100 points) or 'full'

# Initialize Alpha Vantage API
ts = TimeSeries(key=API_KEY, output_format='pandas')

In [33]:
try:
    data, meta_data = ts.get_intraday(
        symbol=SYMBOL,
        interval=INTERVAL,
        outputsize=OUTPUT_SIZE
    )
    
    print("Latest Data Points:")
    print(data.tail())  # Show most recent 5 entries
    
    print("\nMetadata:")
    print(meta_data)

except ValueError as e:
    print(f"Error fetching data: {e}")

Error fetching data: Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.


In [ ]:
def get_alpha_vantage_data(api_key, symbol, interval):
    url = (
        f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY"
        f"&symbol={symbol}&interval={interval}&apikey={api_key}"
    )
    try:
        response = requests.get(url)
        response.raise_for_status()  # will raise an exception for HTTP errors
        data = response.json()
        # You might want to add error-checking if the API returns an error message
        return data
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None


Raw API Response:
{
  "Information": "Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits."
}


ValueError: No data received from Alpha Vantage

In [ ]:
while True:
    data_points = get_alpha_vantage_data()
    for timestamp, data in data_points.items():
        producer.send('stock-prices', value=data)
        print(data)
    time.sleep(1)  